In [ ]:
!pip install -q boto3
!pip install -q xgboost==0.90
!pip install -q scikit-learn==0.20.3
!pip install -q nltk==3.4.5

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

#sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
!head ./data/amazon90000.tsv

In [ ]:
df = pd.read_csv('./data/amazon90000.tsv', delimiter='\t')
df.shape

In [ ]:
df.head(5)

# Clean commas from raw text

In [ ]:
df_scrubbed = df

df_scrubbed['marketplace'] = df_scrubbed['marketplace'].replace(',', ' ')
df_scrubbed['review_id'] = df_scrubbed['review_id'].replace(',', ' ')
df_scrubbed['product_id'] = df_scrubbed['product_id'].replace(',', ' ')
df_scrubbed['product_title'] = df_scrubbed['product_title'].replace(',', ' ')
df_scrubbed['product_category'] = df_scrubbed['product_category'].replace(',', ' ')
df_scrubbed['review_headline'] = df_scrubbed['review_headline'].replace(',', ' ')
df_scrubbed['review_body'] = df_scrubbed['review_body'].replace(',', ' ')
df_scrubbed['review_date'] = df_scrubbed['review_date'].replace(',', ' ')

df_scrubbed.shape

In [ ]:
df_scrubbed.head(5)

# Drop NaNs

In [ ]:
df_scrubbed.isna().values.any()

In [ ]:
df_scrubbed = df.dropna()
df_scrubbed = df_scrubbed.reset_index(drop=True)
df_scrubbed.shape

In [ ]:
df_scrubbed.head(5)

In [ ]:
df_scrubbed['is_positive_sentiment'] = (df_scrubbed['star_rating'] >= 4).astype(int)
df_scrubbed.shape

In [ ]:
df_scrubbed.head(5)

# Write the data file locally

In [ ]:
# $S3_BUCKET/feature-store/amazon-reviews/scrubbed-raw-with-header/data.csv

prefix = 'feature-store/amazon-reviews/scrubbed-raw-with-header'

scrubbed_raw_with_header_path = './{}/data.csv'.format(prefix)

import os
os.makedirs(prefix, exist_ok=True)

df_scrubbed.to_csv(scrubbed_raw_with_header_path, index=False, header=True)


# Upload to S3

In [ ]:
df_scrubbed_raw_with_header_s3_uri = sess.upload_data(path=scrubbed_raw_with_header_path, key_prefix=prefix)

print(df_scrubbed_raw_with_header_s3_uri)


In [ ]:
import seaborn as sns

sns.countplot(x='is_positive_sentiment', data=df_scrubbed)


# Balance the Dataset between Classes

In [ ]:
from sklearn.utils import resample

is_negative_sentiment_df = df_scrubbed.query('is_positive_sentiment == 0')
is_positive_sentiment_df = df_scrubbed.query('is_positive_sentiment == 1')

# TODO:  check which sentiment has the least number of samples

is_positive_downsampled_df = resample(is_positive_sentiment_df,
                                      replace = False,
                                      n_samples = len(is_negative_sentiment_df),
                                      random_state = 27)

df_balanced_raw = pd.concat([is_negative_sentiment_df, is_positive_downsampled_df])
df_balanced_raw = df_balanced_raw.reset_index(drop=True)

In [ ]:
df_balanced_raw.head(5)

In [ ]:
sns.countplot(x='is_positive_sentiment', data=df_balanced_raw)


In [ ]:
df_balanced_raw_only_2_columns = df_balanced_raw[['is_positive_sentiment', 'review_body']]
#df_y_balanced_raw = df_balanced_raw['is_positive_sentiment']

df_balanced_raw_only_2_columns.shape

# Write the data file locally

In [ ]:
# $S3_BUCKET/feature-store/amazon-reviews/balanced-raw-with-header/data.csv

prefix = 'feature-store/amazon-reviews/balanced-raw-with-header'

balanced_raw_with_header_path = './{}/data.csv'.format(prefix)

import os
os.makedirs(prefix, exist_ok=True)

df_balanced_raw_only_2_columns.to_csv(balanced_raw_with_header_path, index=False, header=True)


# Upload to S3

In [ ]:
df_balanced_raw_with_header_s3_uri = sess.upload_data(path=balanced_raw_with_header_path, key_prefix=prefix)

print(df_balanced_raw_with_header_s3_uri)


# TF/IDF Feature Engineering

# Transform the raw text into TF/IDF features

In [ ]:
# Features (X)
df_X_balanced_raw = df_balanced_raw[['review_body']]
# Labels (y)
df_y_balanced_raw = df_balanced_raw['is_positive_sentiment']

print('X.shape:  {}'.format(df_X_balanced_raw.shape))
print('y.shape:  {}'.format(df_y_balanced_raw.shape))

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

def feature_transform_fn(df_text):
    text_processors = Pipeline(
        steps=[
            (
                'tfidfvectorizer',
                TfidfVectorizer(
                    max_df=0.25,                                       
                    min_df=.0025,
                    analyzer='word',
                    max_features=10000
                )
            )
        ]
    )

    column_transformer = ColumnTransformer(
        transformers=[('text_processing', text_processors, df_text.columns.get_loc('review_body'))]
    )

    pipeline = Pipeline(
        steps=[
            ('column_transformer',
             column_transformer), ('svd', TruncatedSVD(n_components=300)),
            ('standardscaler', StandardScaler())
        ]
    )

    return pipeline

In [ ]:
np_tfidf = feature_transform_fn(df_X_balanced_raw).fit_transform(df_X_balanced_raw)
df_tfidf = pd.DataFrame(np_tfidf)
df_tfidf.shape

In [ ]:
df_tfidf.head(5)

### Based on the following:  https://aws.amazon.com/blogs/machine-learning/flagging-suspicious-healthcare-claims-with-amazon-sagemaker/

_Note:  This takes a while.  Please be patient._

In [ ]:
from sklearn.manifold import TSNE

tsne_model = TSNE(perplexity=10, n_components=2, init='pca', n_iter=250, random_state=10)
new_values = tsne_model.fit_transform(new_df_tfidf)


In [ ]:
import matplotlib.pyplot as plt

# column 0-299
labels = df_tfidf.columns.values
x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])

plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    plt.scatter(x[i],y[i])
    plt.annotate(labels[i],
                 xy=(x[i], y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
plt.show()

In [ ]:
#vectorizer = TfidfVectorizer(stop_words='english')
#df_tfidf = vectorizer.fit_transform(text)

#feature_names = vectorizer.get_feature_names()
#print(feature_names)

#df_tfidf = pd.DataFrame(vectorizer.idf_, 
#                        index=vectorizer.get_feature_names(),
#                        columns=['idf'])

#df_tfidf = pd.DataFrame(X.todense(),columns=vectorizer.get_feature_names())
#df_tfidf.head(5)

# Transform just the features (X)
_This will run for a minute or two.  Please be patient._

In [ ]:
np_tfidf = transformer.fit_transform(df_X_balanced_raw)
df_tfidf = pd.DataFrame(np_tfidf)
df_tfidf.shape

In [ ]:
df_tfidf.head(5)

# Add back the label (y) into the first column
The label needs to be in the 1st column for some of our models.

In [ ]:
df_tfidf.insert(0, 'is_positive_sentiment', df_y_balanced_raw)
df_tfidf.shape

In [ ]:
df_tfidf.head(10)

# Randomly split the data into `train`, `validation`, and `test` datasets

In [ ]:
from sklearn.model_selection import train_test_split

# Split all data into 90% train and 10% holdout
df_tfidf_train, df_tfidf_holdout = train_test_split(df_tfidf, test_size=0.1, random_state=0)

# Split the holdout into 50% validation and 50% test
df_tfidf_validation, df_tfidf_test = train_test_split(df_tfidf_holdout, test_size=0.5, random_state=0)

print('df_tfidf.shape={}'.format(df_tfidf.shape))
print('df_tfidf_train.shape={}'.format(df_tfidf_train.shape))
print('df_tfidf_validation.shape={}'.format(df_tfidf_validation.shape))
print('df_tfidf_test.shape={}'.format(df_tfidf_test.shape))

In [ ]:
df_tfidf_train.head(5)

# Write the datasets locally
_Note: `header=False`_

In [ ]:
# $S3_BUCKET/feature-store/amazon-reviews/balanced-tfidf-without-header/data.csv

prefix_train = 'feature-store/amazon-reviews/balanced-tfidf-without-header/train'
prefix_validation = 'feature-store/amazon-reviews/balanced-tfidf-without-header/validation'
prefix_test = 'feature-store/amazon-reviews/balanced-tfidf-without-header/test'

balanced_tfidf_without_header_train_path = './{}/data.csv'.format(prefix_train)
balanced_tfidf_without_header_validation_path = './{}/data.csv'.format(prefix_validation)
balanced_tfidf_without_header_test_path = './{}/data.csv'.format(prefix_test)

import os
os.makedirs(prefix_train, exist_ok=True)
os.makedirs(prefix_validation, exist_ok=True)
os.makedirs(prefix_test, exist_ok=True)

df_tfidf_train.to_csv(balanced_tfidf_without_header_train_path, index=False, header=False)
df_tfidf_validation.to_csv(balanced_tfidf_without_header_validation_path, index=False, header=False)
df_tfidf_test.to_csv(balanced_tfidf_without_header_test_path, index=False, header=False)


# Upload to S3

In [ ]:
df_balanced_tfidf_without_header_train_s3_uri = sess.upload_data(path=balanced_tfidf_without_header_train_path, key_prefix=prefix_train)
df_balanced_tfidf_without_header_validation_s3_uri = sess.upload_data(path=balanced_tfidf_without_header_train_path, key_prefix=prefix_validation)
df_balanced_tfidf_without_header_test_s3_uri = sess.upload_data(path=balanced_tfidf_without_header_train_path, key_prefix=prefix_test)

print(df_balanced_tfidf_without_header_train_s3_uri)
print(df_balanced_tfidf_without_header_validation_s3_uri)
print(df_balanced_tfidf_without_header_test_s3_uri)


In [ ]:
!aws s3 ls $df_balanced_tfidf_without_header_train_s3_uri